In [ ]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import pandas as pd
import sys

In [ ]:
url = 'https://www.seloger.com/immobilier/achat/immo-montigny-les-cormeilles-95/bien-appartement/'

In [ ]:
def getSoup(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text,"html.parser")
    return soup

In [ ]:
def getNumberOfPages(url):
    soup = getSoup(url)
    div = soup.findAll('div', {"class": "Status__StatusContainer-yh17cz-0 ckWPHD"})
    for element in div:
        numberOfElements = str(element).split("-->")[-1].split("</div>")[0]
    numberOfPages = int(numberOfElements)/25
    if numberOfPages > int(numberOfPages):
        numberOfPages = int(numberOfPages) +1
    else:
        numberOfPages = int(numberOfPages)
    return numberOfPages

In [ ]:
getNumberOfPages(url)

In [ ]:
def iterateThroughPages(url):
    #https://www.seloger.com/immobilier/achat/immo-montigny-les-cormeilles-95/bien-appartement/?LISTING-LISTpg=2

In [ ]:
def getListOfUrl(url):
    l = []
    soup = getSoup(url)
    listOfUrl = soup.findAll('a', {"z": "1", "name": "classified-link"})
    for link in listOfUrl:
        l.append(link.get("href"))
    return l

In [ ]:
listOfAnnonce = getListOfUrl(url)

In [ ]:
listOfAnnonce

In [ ]:
#soup = getSoup(listOfAnnonce[4])
#soup = getSoup("https://www.seloger.com/annonces/achat/appartement/montigny-les-cormeilles-95/154352915.htm")

In [ ]:
def getPrix(soup):
    try:
        prix = soup.findAll('span',{"class": "global-styles__TextNoWrap-sc-1aeotog-7 iAJdaY"})[0].string
        prix = int(prix.replace("€","").replace(" ",""))
        return prix
    except:
        print("Error getting price:", sys.exc_info()[0])
        return None

def getSurface(soup):
    try:
        surface = soup.findAll('div',{"class": "Tags__TagContainer-sc-1gccxml-0 kQEoGa"})[-1].string
        surface = float(surface.split("m")[0].replace(",","."))
        return surface
    except:
        print("Error getting surface:", sys.exc_info()[0])
        return None

def getPieces(soup):
    try:
        pieces = soup.findAll('div',{"class": "Tags__TagContainer-sc-1gccxml-0 kQEoGa"})[0].string
        pieces = int(pieces.split(" ")[0])
        return pieces
    except:
        print("Error getting pieces:", sys.exc_info()[0])
        return None

def getChambres(soup):
    try:
        chambres = soup.findAll('div',{"class": "Tags__TagContainer-sc-1gccxml-0 kQEoGa"})[1].string
        chambres = int(chambres.split(" ")[0])
        return chambres
    except:
        print("Error getting chambres:", sys.exc_info()[0])
        return None

def getLots(description):
    try:
        if "Copropriété de " in description:
            lots = int(description.split("Copropriété de ")[1].split(" ")[0])
            return lots
        else:
            return None
    except:
        print("Error getting lots:", sys.exc_info()[0])
        return None
    
def getCharges(description):
    try:
        if "Charges annuelles: " in description:
            charges = float(description.split("Charges annuelles: ")[1].split(" ")[0])
            return charges
        else:
            return None
    except:
        print("Error getting charges:", sys.exc_info()[0])
        return None

def getDescription(soup):
    try:
        description = soup.find('p').string
        return description
    except:
        print("Error getting description:", sys.exc_info()[0])
        return None

def getPlusList(soup):
    l = []
    try :
        plusList = soup.find('div',{"data-test": "plus-list"})
        plusList = plusList.findAll('figcaption')
        for element in plusList:
            l.append(element.string)
        return l
    except:
        print("Error getting plus list:", sys.exc_info()[0])
        return None
    
def getGeneralList(soup):
    l = []
    try :
        generalList = soup.findAll('li',{"class": "GeneralList__Item-sc-9gtpjm-1 dsevYN"})
        for element in generalList:
            l.append(element.string)
        return l
    except:
        print("Error getting general list:", sys.exc_info()[0])
        return None
    
def getListOfData(soup):
    l = []
    #try:
    div_summary = soup.find('div',{"data-test": "summary"})
        
    prix = getPrix(div_summary)
    l.append(prix)
        
    surface = getSurface(div_summary)
    l.append(surface)
        
    pieces = getPieces(div_summary)
    l.append(pieces)
        
    chambres = getChambres(div_summary)
    l.append(chambres)
    
    description = getDescription(soup)
    l.append(description)
        
    lots = getLots(description)
    l.append(lots)
        
    charges = getCharges(description)
    l.append(charges)
            
    plusList = getPlusList(soup)
    l.append(plusList)
    
    generalList = getGeneralList(soup)
    l.append(generalList)
        
    return l
    #except:
        #print("Unexpected error:", sys.exc_info()[0])
        #return [None, None, None, None, None, None, None]

In [ ]:
#getListOfData(soup)

In [ ]:
l = []
for annonce in listOfAnnonce:
    print(annonce)
    soup = getSoup(annonce)
    listOfData = getListOfData(soup)
    listOfData.append(annonce)
    l.append(listOfData)
    time.sleep(1)

In [ ]:
#l

In [ ]:
cols=['prix', 'surface', 'pieces', 'chambres', 'description', 'lots', 'charges', 'plusList', 'generalList', 'url']
df = pd.DataFrame.from_records(l, columns=cols)

In [ ]:
def cleanDf(df):
    return df.dropna(subset=["prix"])

In [ ]:
dfCleaned = cleanDf(df)
dfCleaned

In [ ]:
def haveCave(row):
    if row["plusList"] is not None:
        for item in row["plusList"]:
            if "CAVE" in item.upper():
                return 1
        return 0
    return None

def haveBalcon(row):
    if row["plusList"] is not None:
        for item in row["plusList"]:
            if "BALCON" in item.upper():
                return 1
        return 0
    return None

def haveAscenseur(row):
    if row["plusList"] is not None:
        for item in row["plusList"]:
            if "ASCENSEUR" in item.upper():
                return 1
        return 0
    return None

In [ ]:
def fromListToColumns(df):
    df['cave'] = df.apply(lambda row: haveCave(row), axis=1)
    df['balcon'] = df.apply(lambda row: haveBalcon(row), axis=1)
    df['ascenseur'] = df.apply(lambda row: haveAscenseur(row), axis=1)
    return df

In [ ]:
df = fromListToColumns(df)

In [ ]:
df